In [ ]:
import pandas as pd
import numpy as np 
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.applications.resnet_v2 import ResNet50V2

from tensorflow.keras.utils import  img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense 
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import tensorflow as tf
%matplotlib inline
import math 
import itertools, re, os, random, keras
import plotly.express as px
import datetime
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import time

In [ ]:
train_path ='insec/train/'
valid_path ='insec/valid/'
test_path ='insec/test/'

In [ ]:

def plotImagesAndLabels(folder):
    c=1
    directory=os.listdir(folder)
    plt.figure(figsize=(28,20))
    for each in directory:
        currentFolder=folder+ "\\" +each
        for i, file in enumerate(os.listdir(currentFolder)[0:4]):
            full_path=currentFolder+"\\"+file
            plt.subplot(3, 8, c)
            img = mpimg.imread(full_path)
            plt.imshow(img)
            plt.title(each)
            c+=1
    plt.subplots_adjust(wspace=0.3, hspace=-0.1)
    plt.show()

In [ ]:
print("Lets see some samples from training data...")
plotImagesAndLabels(train_path)

In [ ]:
#Default dimensions we found online
img_width, img_height = 224, 224 
 
#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model_ResNet50V2.h5'

 
# number of epochs to train top model 
epochs = 15 #this has been changed after multiple model run 
# batch size used by flow_from_directory and predict_generator 
batch_size = 30

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import ResNet50V2


ResNet50 = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet')
for layer in ResNet50.layers:
        layer.trainable=False
datagen = ImageDataGenerator(rescale=1. / 255) 
#needed to create the bottleneck .npy files

In [ ]:
#__this can take an hour and half to run so only run it once. 
#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__
from keras.applications.resnet_v2 import ResNet50V2

start = datetime.datetime.now()
 
generator = datagen.flow_from_directory( 
    train_path, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_train_samples = len(generator.filenames) 
num_classes = len(generator.class_indices) 
 
predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 
 
bottleneck_features_train = ResNet50.predict_generator(generator, predict_size_train) 
 
np.save('bottleneck_features_train_ResNet50V2.npy', bottleneck_features_train)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
start = datetime.datetime.now()
 
generator = datagen.flow_from_directory( 
   valid_path, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_train_samples = len(generator.filenames) 
num_classes = len(generator.class_indices) 
 
predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 
 
bottleneck_features_valid = ResNet50.predict_generator(generator, predict_size_train) 
 
np.save('bottleneck_features_valid_ResNet50.npy', bottleneck_features_valid)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
start = datetime.datetime.now()
 
generator = datagen.flow_from_directory( 
   test_path, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_train_samples = len(generator.filenames) 

num_classes = len(generator.class_indices) 
 
predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 
 
bottleneck_features_test = ResNet50.predict_generator(generator, predict_size_train) 
 
np.save('bottleneck_features_test_ResNet50.npy', bottleneck_features_test)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
#training data
generator_top = datagen.flow_from_directory( 
   train_path, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
train_data = np.load('bottleneck_features_train_ResNet50V2.npy') 
 
# get the class labels for the training data, in the original order 
train_labels = generator_top.classes 
 
# convert the training labels to categorical vectors P
train_labels = to_categorical(train_labels, num_classes=num_classes)

In [ ]:
generator_top = datagen.flow_from_directory( 
   valid_path, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
valid_data = np.load('bottleneck_features_valid_ResNet50.npy') 
 
# get the class labels for the training data, in the original order 
valid_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
valid_labels = to_categorical(valid_labels, num_classes=num_classes)

In [ ]:
generator_top = datagen.flow_from_directory( 
   test_path, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
test_data = np.load('bottleneck_features_test_ResNet50.npy') 
 
# get the class labels for the training data, in the original order 
test_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image

#This is the best model we found.
start = datetime.datetime.now()
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(224,
                                  224,
                                  3)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
    layers.RandomTranslation(0.2, 0.2),
  ]
)

model = Sequential() 
model.add(Flatten(input_shape=train_data.shape[1:])) 
model.add(Dense(200, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.75)) 
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5)) 
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
   optimizer=optimizers.RMSprop(lr=1e-4),
   metrics=['acc'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('resnet.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True, patience=10)

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

history = model.fit(train_datagen.flow(train_data, train_labels, batch_size=batch_size),
   epochs=10,
   batch_size=batch_size, 
   validation_data=(valid_data, valid_labels),
   callbacks=[es,mc])
model.save_weights(top_model_weights_path)

(eval_loss, eval_accuracy) = model.evaluate( 
    valid_data, valid_labels, batch_size=batch_size, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("[INFO] Loss: {}".format(eval_loss)) 

end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

In [ ]:
#Graphing our training and validation
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy') 
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss') 
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_data,test_labels)

In [ ]:
preds=np.round(model.predict(test_data),0)
print('rounded test_labels',preds)

In [ ]:
birds=['Bees', 'Beetles', 'Buttefly', 'Cicada','Dragonfly','Grasshopper','Ladybird']
classification_metrics=metrics.classification_report(test_labels,preds,target_names=birds)
print(classification_metrics)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

categorical_test_labels = pd.DataFrame(test_labels).idxmax(axis=1)
categorical_preds = pd.DataFrame(preds).idxmax(axis=1)
conf_mat = confusion_matrix(categorical_test_labels, categorical_preds)

#To get better visual of the confusion matrix:
def plot_confusion_matrix(cm, classes,
   normalize=False,
   title='Confusion matrix',
   cmap=plt.cm.Blues):
 
#Add Normalization Option
 
   if normalize:
     cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
     print("Normalized confusion matrix")
   else:
     print('Confusion matrix, without normalization')
 
# print(cm)
 
   plt.imshow(cm, interpolation='nearest', cmap=cmap)
   plt.title(title)
   plt.colorbar()
   tick_marks = np.arange(len(classes))
   plt.xticks(tick_marks, classes, rotation=45)
   plt.yticks(tick_marks, classes)
 
   fmt = '.2f' if normalize else 'd'
   thresh = cm.max() / 2.
   for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
 
   plt.tight_layout()
   plt.ylabel('True label')
   plt.xlabel('Predicted label')

plot_confusion_matrix(conf_mat, classes = ['Bees', 'Beetles', 'Buttefly', 'Cicada', 'Dragonfly', 'Grasshopper', 'Ladybird'])